In [1]:
import altair as alt
import pandas as pd


# FBI Data

In [34]:
df = pd.read_csv('../data/hate_crime_fbi.csv')

alt.data_transformers.disable_max_rows()

df.columns

Index(['incident_id', 'data_year', 'ori', 'pug_agency_name', 'pub_agency_unit',
       'agency_type_name', 'state_abbr', 'state_name', 'division_name',
       'region_name', 'population_group_code', 'population_group_description',
       'incident_date', 'adult_victim_count', 'juvenile_victim_count',
       'total_offender_count', 'adult_offender_count',
       'juvenile_offender_count', 'offender_race', 'offender_ethnicity',
       'victim_count', 'offense_name', 'total_individual_victims',
       'location_name', 'bias_desc', 'victim_types', 'multiple_offense',
       'multiple_bias'],
      dtype='object')

In [35]:
df['bias_desc'].unique()

array(['Anti-Black or African American', 'Anti-White', 'Anti-Jewish',
       'Anti-Arab', 'Anti-Protestant', 'Anti-Other Religion',
       'Anti-Islamic (Muslim)', 'Anti-Gay (Male)', 'Anti-Asian',
       'Anti-Catholic', 'Anti-Multiple Religions, Group',
       'Anti-Hispanic or Latino', 'Anti-Multiple Races, Group',
       'Anti-Lesbian (Female)', 'Anti-Other Race/Ethnicity/Ancestry',
       'Anti-Heterosexual',
       'Anti-Lesbian, Gay, Bisexual, or Transgender (Mixed Group)',
       'Anti-American Indian or Alaska Native',
       'Anti-Gay (Male);Anti-White',
       'Anti-Black or African American;Anti-Jewish',
       'Anti-Black or African American;Anti-Lesbian (Female)',
       'Anti-Black or African American;Anti-Gay (Male)',
       'Anti-Black or African American;Anti-White',
       'Anti-Atheism/Agnosticism', 'Anti-Gay (Male);Anti-Jewish',
       'Anti-Bisexual', 'Anti-Hispanic or Latino;Anti-White',
       'Anti-Hispanic or Latino;Anti-Multiple Races, Group',
       'Anti-Ame

In [ ]:
# Filtering by keywords
black_df = df[df['bias_desc'].str.contains('Anti-Black', case=False, na=False)].copy()
hispanic_df = df[df['bias_desc'].str.contains('Anti-Hispanic', case=False, na=False)].copy()
asian_df = df[df['bias_desc'].str.contains('Anti-Asian', case=False, na=False)].copy()

# Assigning labels to each category
black_df['group'] = 'Anti-Black'
hispanic_df['group'] = 'Anti-Hispanic'
asian_df['group'] = 'Anti-Asian'

# Combining the filtered DataFrames
combined_df = pd.concat([black_df, hispanic_df, asian_df])

# Aggregating the number of incidents per year
counts_by_year = (
    combined_df.groupby(['data_year', 'group'])
    .size()
    .reset_index(name='count')
)

# Plotting the chart with Altair
chart = alt.Chart(counts_by_year).mark_line(point=True).encode(
    x=alt.X('data_year:O', title='Year'),
    y=alt.Y('count:Q', title='Number of Incidents'),
    color=alt.Color('group:N', title='Bias Type')
).properties(
    width=600,
    height=400,
    title='Hate Crime Incidents by Bias Type and Year'
)

chart

alt.Chart(...)

In [ ]:
# Filter by keywords
black_df = df[df['bias_desc'].str.contains('Anti-Black', case=False, na=False)].copy()
hispanic_df = df[df['bias_desc'].str.contains('Anti-Hispanic', case=False, na=False)].copy()
asian_df = df[df['bias_desc'].str.contains('Anti-Asian', case=False, na=False)].copy()

# Assign labels to each category
black_df['group'] = 'Anti-Black'
hispanic_df['group'] = 'Anti-Hispanic'
asian_df['group'] = 'Anti-Asian'

# Combine the filtered DataFrames
combined_df = pd.concat([black_df, hispanic_df, asian_df])

# Aggregate the number of incidents per year
counts_by_year = (
    combined_df.groupby(['data_year', 'group'])
    .size()
    .reset_index(name='count')
)

# Population by race/ethnicity (based on 2020 Census, fixed)
population_dict = {
    'Anti-Black': 41288572,
    'Anti-Hispanic': 61755866,
    'Anti-Asian': 19112979
}

# Map population and calculate incidents per 100,000 people
counts_by_year['population'] = counts_by_year['group'].map(population_dict)
counts_by_year['per_100k'] = counts_by_year['count'] / counts_by_year['population'] * 100000

# Draw the chart with Altair (per capita basis)
chart = alt.Chart(counts_by_year).mark_line(point=True).encode(
    x=alt.X('data_year:O', title='Year'),
    y=alt.Y('per_100k:Q', title='Incidents per 100,000 People'),
    color=alt.Color('group:N', title='Bias Type'),
    tooltip=['data_year', 'group', 'per_100k']
).properties(
    width=600,
    height=400,
    title='Hate Crime Incidents per 100,000 People by Bias Type and Year'
)

chart

alt.Chart(...)

In [9]:
asian_df = df[df['bias_desc'] == 'Anti-Asian']

asian_df.iloc[0].T


incident_id                                                     33
data_year                                                     1991
ori                                                      AZ0072300
pug_agency_name                                            Phoenix
pub_agency_unit                                                NaN
agency_type_name                                              City
state_abbr                                                      AZ
state_name                                                 Arizona
division_name                                             Mountain
region_name                                                   West
population_group_code                                           1B
population_group_description      Cities from 500,000 thru 999,999
incident_date                                           1991-11-13
adult_victim_count                                             NaN
juvenile_victim_count                                         

In [ ]:
# Columns to compare
columns_to_compare = ['victim_types', 'location_name', 'offender_race']

# Dictionary to store the results
results = {}

# Filter by each bias group
groups = {
    'Anti-Black': df[df['bias_desc'].str.contains('Anti-Black', case=False, na=False)].copy(),
    'Anti-Hispanic': df[df['bias_desc'].str.contains('Anti-Hispanic', case=False, na=False)].copy(),
    'Anti-Asian': df[df['bias_desc'].str.contains('Anti-Asian', case=False, na=False)].copy()
}

# Aggregate for each column
for col in columns_to_compare:
    frames = []
    for group_name, group_df in groups.items():
        # Calculate percentages
        count = group_df[col].value_counts(dropna=False, normalize=True) * 100
        count_df = count.rename(f'{group_name} (%)').reset_index()
        count_df.columns = [col, f'{group_name} (%)']
        frames.append(count_df.set_index(col))
    
    # Join horizontally
    comparison_df = pd.concat(frames, axis=1).fillna(0).reset_index()
    results[col] = comparison_df

# Display results by category (uncomment as needed)

print("=== Comparison by victim_types ===")
print(results['victim_types'].to_string(index=False))
print("\n=== Comparison by location_name ===")
print(results['location_name'].to_string(index=False))
print("\n=== Comparison by offender_race ===")
print(results['offender_race'].to_string(index=False))

=== Comparison by victim_types ===
                                         victim_types  Anti-Black (%)  Anti-Hispanic (%)  Anti-Asian (%)
                                           Individual       82.339125          92.806932       87.256809
                                                Other        5.215064           1.811902        3.793774
                                             Business        3.847907           1.872500        4.231518
                                           Government        3.726381           0.727185        1.179475
                                       Society/Public        2.117342           0.842322        1.009241
                                              Unknown        0.583087           0.157557        0.182393
                               Religious Organization        0.566728           0.193916        0.522860
                                     Individual;Other        0.420664           0.418131        0.389105
                    

# ICPSR Data Processing

In [ ]:
# Path to the TSV file
tsv_file = '../data/ICPSR_2023/DS0003/39270-0003-Data.tsv'

# List to store filtered results
filtered_chunks = []

# Number of rows per chunk (adjust depending on memory capacity)
chunk_size = 500000

# Read the TSV file in chunks
for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
    
    # Filter rows where V20201 is not '88' (i.e., bias motivation is not "None")
    filtered_chunk = chunk[chunk['V20201'] != '88']
    
    # Append filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
df_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Save the filtered data to a lightweight CSV file
df_filtered.to_csv('2023-03.csv', index=False)

/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_4397/3597679678.py:4: DtypeWarning: Columns (22,62,222,231,232,233,351,352,353) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bias = pd.read_csv('39270-0003-Data_bias.csv')


   SEGMENT  STATE  FIPS_STATE        ORI    INCNUM      INCDATE  RECSBH  \
0        1     50           2  AK0010200  23000233  15-JAN-2023       1   
1        1     50           2  AK0010200  23000798  25-FEB-2023       1   
2        1     50           2  AK0010200  23001158  26-MAR-2023       1   
3        1     50           2  AK0010200  23003298  27-AUG-2023       1   
4        1     50           2  AK0010200  23003674  21-SEP-2023       1   

   RECSADM  RECSOFS  RECSPRP  ...  V60171  V60172  V60173 V60181 V60182  \
0        1        1        1  ...                                         
1        1        1        0  ...       1                      1          
2        1        1        0  ...       0                      0          
3        1        1        0  ...       0                      1          
4        1        1        1  ...                                         

  V60183 V60191  V60192  V60193  ALLOFNS  
0                                    290  
1           

In [ ]:
# Path to the TSV file
tsv_file = '../data/ICPSR_2022/DS0003/38925-0003-Data.tsv'

# List to store filtered results
filtered_chunks = []

# Number of rows per chunk (adjust depending on memory capacity)
chunk_size = 500000

# Read the TSV file in chunks
for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
    
    # Filter rows where V20201 is not '88' (i.e., bias motivation is not "None")
    filtered_chunk = chunk[chunk['V20201'] != '88']
    
    # Append filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
df_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Save the filtered data to a lightweight CSV file
df_filtered.to_csv('2022-03.csv', index=False)

/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/1918523034.py:11: DtypeWarning: Columns (25,32,33,36,37,62,63,64,192,222,231,232,233,342,351,352,353) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/1918523034.py:11: DtypeWarning: Columns (62,192,222,231,232,233,351,352,353) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/1918523034.py:11: DtypeWarning: Columns (32,33,36,37,62,63,64,135,138,141,192,222,231,232,233,351,352,353) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wp

   SEGMENT  STATE  FIPS_STATE        ORI        INCNUM      INCDATE  RECSBH  \
0        1     50           2  AK0010400  CT-4-C5U728N  21-MAR-2022       1   
1        1     50           2  AK0010500  28-HY-O2TD1A  31-JUL-2022       1   
2        1     50           2  AK0010800  CT-BRMQU728N  07-APR-2022       1   
3        1     50           2  AK0011000  CT0VRCSC-A8N  05-MAY-2022       1   
4        1     50           2  AK0011300  CT01KV3U728N  26-FEB-2022       1   

   RECSADM  RECSOFS  RECSPRP  ...  V60171  V60172  V60173 V60181 V60182  \
0        1        1        1  ...                                         
1        1        1        1  ...                                         
2        1        1        0  ...      -7                      0          
3        1        1        1  ...                                         
4        1        1        1  ...                                         

  V60183 V60191  V60192  V60193  ALLOFNS  
0                              

In [ ]:
# Path to the TSV file
tsv_file = '../data/ICPSR_2021/DS0003/38807-0003-Data.tsv'

# List to store filtered results
filtered_chunks = []

# Number of rows per chunk (adjust depending on memory capacity)
chunk_size = 500000

# Read the TSV file in chunks
for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
    
    # Filter rows where V20201 is not '88' (i.e., bias motivation is not "None")
    filtered_chunk = chunk[chunk['V20201'] != '88']
    
    # Append filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
df_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Save the filtered data to a lightweight CSV file
df_filtered.to_csv('2021-03.csv', index=False)

/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/2586642471.py:11: DtypeWarning: Columns (25,32,33,62,63,192,222,231,232,233,342,351,352,353) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/2586642471.py:11: DtypeWarning: Columns (32,33,36,37,62,63,64,135,192,222,231,232,233,351,352,353) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/2586642471.py:11: DtypeWarning: Columns (32,33,62,63,135,222,231,232,233,339,342,351,352,353,390) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/5

   SEGMENT  STATE  FIPS_STATE        ORI        INCNUM      INCDATE  RECSBH  \
0        1     50           2  AK0010200  CN0BAM5D728N  02-AUG-2021       1   
1        1     50           2  AK0010200  CN0BIVMR728N  13-AUG-2021       1   
2        1     50           2  AK0010200  CN0BIWS9728N  24-SEP-2021       1   
3        1     50           2  AK0010400  CN-VAO5U728N  03-OCT-2021       1   
4        1     50           2  AK0010600  3I-MQ3QCZX I  18-JUL-2021       1   

   RECSADM  RECSOFS  RECSPRP  ...  V60171  V60172  V60173 V60181 V60182  \
0        1        1        0  ...                                         
1        1        1        0  ...                                         
2        1        1        1  ...                                         
3        1        1        1  ...                                         
4        1        2        3  ...                                         

  V60183 V60191  V60192  V60193  ALLOFNS  
0                              

In [ ]:
# Path to the TSV file
tsv_file = '../data/ICPSR_2020/DS0003/38566-0003-Data.tsv'

# List to store filtered results
filtered_chunks = []

# Number of rows per chunk (adjust depending on memory capacity)
chunk_size = 500000

# Read the TSV file in chunks
for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
    
    # Filter rows where V20201 is not '88' (i.e., bias motivation is not "None")
    filtered_chunk = chunk[chunk['V20201'] != '88']
    
    # Append filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
df_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Save the filtered data to a lightweight CSV file
df_filtered.to_csv('2020-03.csv', index=False)

/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/3992448913.py:11: DtypeWarning: Columns (21,31,32,35,36,62,63,134,221,230,231,232,335,341,352) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/3992448913.py:11: DtypeWarning: Columns (27,31,32,62,134,221,230,231,232,335,341,351,352,389) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/3992448913.py:11: DtypeWarning: Columns (27,31,32,62,221,230,231,232,335,341,352) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76c

   SEGMENT  STATE        ORI        INCNUM      INCDATE  RECSBH  RECSADM  \
0        1      1  AL0010700  6X30VE32UIL8  08-MAR-2020       1        1   
1        1      1  AL0010700  B3BYFX E0WXQ  27-APR-2020       1        1   
2        1      1  AL0010700  T0C03FQPFAA0  14-APR-2020       1        1   
3        1      1  AL0010700  ZD1M0Q5OMF02  13-MAY-2020       1        1   
4        1      1  AL0011200  2W2HPU7 6XKI  03-JUN-2020       1        1   

   RECSOFS  RECSPRP  RECSVIC  ...  V60171  V60172 V60173 V60181 V60182 V60183  \
0        1        1        1  ...                                               
1        1        1        1  ...                                               
2        1        1        1  ...                                               
3        1        1        1  ...                                               
4        1        1        1  ...                                               

   V60191  V60192  V60193  ALLOFNS  
0                  

In [ ]:
# Path to the TSV file
tsv_file = '../data/ICPSR_2019/DS0003/38565-0003-Data.tsv'

# List to store filtered results
filtered_chunks = []

# Number of rows per chunk (adjust depending on memory capacity)
chunk_size = 500000

# Read the TSV file in chunks
for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
    
    # Filter rows where V20201 is not '88' (i.e., bias motivation is not "None")
    filtered_chunk = chunk[chunk['V20201'] != '88']
    
    # Append filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
df_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Save the filtered data to a lightweight CSV file
df_filtered.to_csv('2019-03.csv', index=False)

/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/2446525155.py:11: DtypeWarning: Columns (27,31,32,35,36,62,63,134,221,230,231,232,335,341,352) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/2446525155.py:11: DtypeWarning: Columns (27,31,32,62,221,230,231,232,335,341,352) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/2446525155.py:11: DtypeWarning: Columns (27,221,230,231,232,335,341,351,352) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/i

   SEGMENT  STATE        ORI        INCNUM      INCDATE  RECSBH  RECSADM  \
0        1      1  AL0011200  2W1WPU7266 I  24-JAN-2019       1        1   
1        1      1  AL0011200  2W1WPU7V8XTX  18-JUN-2019       1        1   
2        1      1  AL0011200  2W1WPUF2JLX4  17-OCT-2019       1        1   
3        1      1  AL0011200  2W1WPUFZ7N0O  27-DEC-2019       1        1   
4        1      1  AL0011200  2W1WPUFZSLTX  19-DEC-2019       1        1   

   RECSOFS  RECSPRP  RECSVIC  ...  V60171  V60172 V60173 V60181 V60182 V60183  \
0        2        2        1  ...                                               
1        1        2        1  ...                                               
2        1        2        1  ...       0                     0                 
3        1        1        1  ...                                               
4        2        1        2  ...       0                     0                 

   V60191  V60192  V60193  ALLOFNS  
0                  

In [10]:
# Path to the TSV file
tsv_file = '../data/ICPSR_2018/DS0003/37649-0003-Data.tsv'

# List to store filtered results
filtered_chunks = []

# Number of rows per chunk (adjust depending on memory capacity)
chunk_size = 500000

# Read the TSV file in chunks
for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
    
    # Filter rows where V20201 is not '88' (i.e., bias motivation is not "None")
    filtered_chunk = chunk[chunk['V20201'] != '88']
    
    # Append filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
df_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Save the filtered data to a lightweight CSV file
df_filtered.to_csv('2018-03.csv', index=False)

/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/4254908074.py:11: DtypeWarning: Columns (27,31,32,35,36,62,63,134,221,230,231,232,335,341,351,352) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/4254908074.py:11: DtypeWarning: Columns (27,31,32,62,221,230,231,232,335,341,352) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/4254908074.py:11: DtypeWarning: Columns (21,24,27,221,230,231,232,335,341,351,352) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c

In [11]:
# Path to the TSV file
tsv_file = '../data/ICPSR_2017/DS0003/37650-0003-Data.tsv'

# List to store filtered results
filtered_chunks = []

# Number of rows per chunk (adjust depending on memory capacity)
chunk_size = 500000

# Read the TSV file in chunks
for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
    
    # Filter rows where V20201 is not '88' (i.e., bias motivation is not "None")
    filtered_chunk = chunk[chunk['V20201'] != '88']
    
    # Append filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
df_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Save the filtered data to a lightweight CSV file
df_filtered.to_csv('2017-03.csv', index=False)

/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/1323432272.py:11: DtypeWarning: Columns (27,31,32,35,36,62,63,134,221,230,231,232,335,341,351,352) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/1323432272.py:11: DtypeWarning: Columns (21,27,31,32,34,45,46,47,48,62,221,230,231,232,352) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/1323432272.py:11: DtypeWarning: Columns (24,27,221,230,231,232,335,341,350,351,352) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52x

In [12]:
# Path to the TSV file
tsv_file = '../data/ICPSR_2016/DS0003/37066-0003-Data.tsv'

# List to store filtered results
filtered_chunks = []

# Number of rows per chunk (adjust depending on memory capacity)
chunk_size = 500000

# Read the TSV file in chunks
for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
    
    # Filter rows where V20201 is not '88' (i.e., bias motivation is not "None")
    filtered_chunk = chunk[chunk['V20201'] != '88']
    
    # Append filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
df_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Save the filtered data to a lightweight CSV file
df_filtered.to_csv('2016-03.csv', index=False)

/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/3314708295.py:11: DtypeWarning: Columns (21,235,236,237) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/3314708295.py:11: DtypeWarning: Columns (21,235,236,237,341,342) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/3314708295.py:11: DtypeWarning: Columns (21,235,236,237,341,342) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/3314708295.py:11: DtypeWarning: Columns (21,235,236,

In [13]:
# Path to the TSV file
tsv_file = '../data/ICPSR_2015/DS0003/36851-0003-Data.tsv'

# List to store filtered results
filtered_chunks = []

# Number of rows per chunk (adjust depending on memory capacity)
chunk_size = 500000

# Read the TSV file in chunks
for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
    
    # Filter rows where V20201 is not '88' (i.e., bias motivation is not "None")
    filtered_chunk = chunk[chunk['V20201'] != '88']
    
    # Append filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
df_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Save the filtered data to a lightweight CSV file
df_filtered.to_csv('2015-03.csv', index=False)

/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/3447377130.py:11: DtypeWarning: Columns (235,236,237,342) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/3447377130.py:11: DtypeWarning: Columns (235,236,237) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/3447377130.py:11: DtypeWarning: Columns (21,235,236,237,341,342) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(tsv_file, sep='\t', chunksize=chunk_size, dtype={'V20201': str}):
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_75740/3447377130.py:11: DtypeWarning: Columns (21,235,236,237,342) h

# Combine

In [ ]:
# Load CSV files from 2023 to 2015 into a list of DataFrames
df_list = [pd.read_csv(f'{year}-03.csv') for year in range(2023, 2014, -1)]

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

# Check the combined DataFrame
print(combined_df.head())
print(combined_df.shape)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('2015-2023_03.csv', index=False)

/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_66097/420286934.py:6: DtypeWarning: Columns (22,62,222,231,232,233,351,352,353) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{year}-03.csv')
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_66097/420286934.py:6: DtypeWarning: Columns (22,62,222,231,232,233,351,352,353) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{year}-03.csv')
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_66097/420286934.py:6: DtypeWarning: Columns (22,62,192,222,231,232,233,351,352,353) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{year}-03.csv')
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_66097/420286934.py:6: DtypeWarning: Columns (27,221,230,231,232,352) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{year}-03.csv')
/var/fo

   SEGMENT  STATE  FIPS_STATE        ORI    INCNUM      INCDATE  RECSBH  \
0        1     50         2.0  AK0010200  23000233  15-JAN-2023       1   
1        1     50         2.0  AK0010200  23000798  25-FEB-2023       1   
2        1     50         2.0  AK0010200  23001158  26-MAR-2023       1   
3        1     50         2.0  AK0010200  23003298  27-AUG-2023       1   
4        1     50         2.0  AK0010200  23003674  21-SEP-2023       1   

   RECSADM  RECSOFS  RECSPRP  ...  V60191  V60192  V60193 ALLOFNS  YEAR V5006  \
0        1        1        1  ...                             290  2023   NaN   
1        1        1        0  ...                             131  2023   NaN   
2        1        1        0  ...                             131  2023   NaN   
3        1        1        0  ...                             131  2023   NaN   
4        1        1        1  ...                             290  2023   NaN   

  V5007  V5008  V5009  V5011  
0   NaN    NaN    NaN    NaN  


# Examine

In [93]:
df = pd.read_csv('2015-2023_03.csv')


/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_66097/2476529448.py:1: DtypeWarning: Columns (5,13,14,22,25,26,28,32,33,36,37,40,41,44,45,62,63,64,65,66,67,74,75,76,79,80,82,83,84,85,86,87,88,89,90,91,92,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293

Focus on each fisrt crime of incidents.

In [ ]:
# Anti-Black=12, Anti-Asian=14, Anti-Hispanic=32
target_codes = [12, 14, 32]
    
race_df = df[df['V20201'].isin(target_codes)]

In [96]:
import pandas as pd
import altair as alt

# Target bias motivation codes and their labels
target_codes = {
    12: 'Anti-Black',
    14: 'Anti-Asian',
    32: 'Anti-Hispanic'
}

# Define a parser that can handle multiple date formats
def parse_date(date_str):
    for fmt in ('%Y%m%d', '%d-%b-%Y', '%Y-%m-%d'):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except (ValueError, TypeError):
            continue
    return pd.NaT

# Clean INCDATE in the entire race_df
race_df['INCDATE'] = race_df['INCDATE'].apply(parse_date)

# Remove rows with unparseable dates
race_df = race_df.dropna(subset=['INCDATE'])

# Extract year from the cleaned date
race_df['YEAR'] = race_df['INCDATE'].dt.year

# Filter rows where V20201 matches any of the target codes
race_df_filtered = race_df[race_df['V20201'].isin(target_codes.keys())].copy()

# Map bias code to label
race_df_filtered['label'] = race_df_filtered['V20201'].map(target_codes)

# Aggregate number of incidents by year and bias label
yearly_counts = (
    race_df_filtered
    .groupby(['YEAR', 'label'])
    .size()
    .reset_index(name='count')
)

# Create Altair line chart
chart = alt.Chart(yearly_counts).mark_line(point=True).encode(
    x=alt.X('YEAR:O', title='Year'),
    y=alt.Y('count:Q', title='Number of Incidents'),
    color=alt.Color('label:N', title='Bias Motivation'),
    tooltip=['YEAR', 'label', 'count']
).properties(
    width=600,
    height=400,
    title='Hate Crime Incidents by Year and Bias Motivation (V20201 only)'
)

chart

alt.Chart(...)

In [98]:
import pandas as pd
import altair as alt

# Ensure INCDATE is already parsed to datetime in race_df
race_df['MONTH'] = race_df['INCDATE'].dt.month

# Target bias motivation codes and their labels
target_codes = {
    12: 'Anti-Black',
    14: 'Anti-Asian',
    32: 'Anti-Hispanic'
}

# Filter for selected bias motivations
race_df_filtered = race_df[race_df['V20201'].isin(target_codes.keys())].copy()
race_df_filtered['label'] = race_df_filtered['V20201'].map(target_codes)

# Count incidents by month and label
monthly_counts = (
    race_df_filtered
    .groupby(['label', 'MONTH'])
    .size()
    .reset_index(name='count')
)

# Calculate percentage by group (label)
monthly_counts['percentage'] = (
    monthly_counts
    .groupby('label')['count']
    .transform(lambda x: x / x.sum() * 100)
)

# Make sure month is categorical for correct ordering
monthly_counts['MONTH'] = monthly_counts['MONTH'].astype(int).astype(str)

# Altair chart: percentage of incidents by month
chart = alt.Chart(monthly_counts).mark_line(point=True).encode(
    x=alt.X('MONTH:O', title='Month'),
    y=alt.Y('percentage:Q', title='Share of Incidents (%)'),
    color=alt.Color('label:N', title='Bias Motivation'),
    tooltip=['label', 'MONTH', 'percentage']
).properties(
    width=600,
    height=400,
    title='Monthly Distribution of Hate Crimes by Bias Motivation (as % of Total)'
)

chart

alt.Chart(...)

### UCR OFFENSE CODE

In [ ]:
# Cross-tabulation (frequency)
offense_bias_table = pd.crosstab(race_df['V20201'], race_df['V20061'])

# Display the result
print(offense_bias_table)

# Cross-tabulation (percentage per V20201)
offense_bias_pct = pd.crosstab(race_df['V20201'], race_df['V20061'], normalize='index') * 100

# Display rounded to two decimal places
print(offense_bias_pct.round(2))

import altair as alt

# Convert data to long format
offense_bias_df = offense_bias_pct.reset_index().melt(
    id_vars='V20201', 
    var_name='V20061', 
    value_name='percentage'
)

# Draw a heatmap with Altair
heatmap = alt.Chart(offense_bias_df).mark_rect().encode(
    x=alt.X('V20061:N', title='UCR Offense Code (V20061)'),
    y=alt.Y('V20201:N', title='Bias Motivation (V20201)'),
    color=alt.Color('percentage:Q', title='Percentage (%)', scale=alt.Scale(scheme='blues')),
    tooltip=['V20201', 'V20061', 'percentage']
).properties(
    title='Bias Motivation by UCR Offense Code',
    width=600,
    height=300
)

heatmap

V20061  91   92   100  111  112  113  114  120   131   132  ...  351  352  \
V20201                                                      ...             
12       17    1   16   11    2    1   25  104  1779  3261  ...  116    7   
14        0    0    3    2    0    0    1   35   141   405  ...   14    1   
33        2    0    3    4    1    0    3   32   192   411  ...   26    1   

V20061  362  370  401  402  403  510  520  720  
V20201                                          
12        1   11    0    1    0    1   84    4  
14        0    0    1    0    1    0    5    0  
33        0    1    0    1    0    0    6    0  

[3 rows x 42 columns]
V20061   91    92    100   111   112   113   114   120    131    132  ...  \
V20201                                                                ...   
12      0.12  0.01  0.11  0.08  0.01  0.01  0.17  0.71  12.18  22.34  ...   
14      0.00  0.00  0.22  0.15  0.00  0.00  0.07  2.58  10.38  29.80  ...   
33      0.10  0.00  0.16  0.21  0.05  

alt.Chart(...)

## Victim

### Victim Sex

In [ ]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V40191'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

V40191    -7    -6     0     1           -6    -7      0      1
V20201                                                         
12      0.12  2.01  4.32  6.75  14.54  0.09  1.05  29.71  41.41
14      0.00  1.03  2.65  4.19   9.93  0.00  0.96  34.44  46.80
32      0.00  0.67  3.09  7.68   4.13  0.00  0.47  30.10  53.86


### Offender Race

In [ ]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V50091'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

NameError: name 'pd' is not defined

### Victim Age

In [79]:
# Convert non-numeric values (e.g., strings) to NaN and then to numeric
race_df['V40181'] = pd.to_numeric(race_df['V40181'], errors='coerce')

# Drop rows with NaN in the age column (exclude missing age data from analysis)
race_df_clean = race_df.dropna(subset=['V40181'])

# Group age into 10-year bins
race_df_clean['age_group'] = pd.cut(
    race_df_clean['V40181'],
    bins=range(0, 101, 10),
    right=False,
    labels=['0-9','10-19','20-29','30-39','40-49',
            '50-59','60-69','70-79','80-89','90-99']
)

# Group by age_group and V20201, then reshape into long format
age_group_counts = (
    race_df_clean.groupby(['V20201', 'age_group'])
    .size()
    .reset_index(name='count')
)

# Calculate the percentage within each Bias Motivation group
age_group_counts['percentage'] = (
    age_group_counts.groupby('V20201')['count']
    .transform(lambda x: x / x.sum() * 100)
)

# Plot with Altair
import altair as alt

chart = alt.Chart(age_group_counts).mark_bar().encode(
    x=alt.X('age_group:N', title='Age Group'),
    y=alt.Y('percentage:Q', title='Percentage (%)'),
    color=alt.Color('age_group:N', legend=None),
    column=alt.Column('V20201:N', title='Bias Motivation (V20201)')
).properties(
    width=100,
    height=300,
    title='Age Distribution by Bias Motivation'
)

chart

/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_66097/4109112118.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_df['V40181'] = pd.to_numeric(race_df['V40181'], errors='coerce')
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_66097/4109112118.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_df_clean['age_group'] = pd.cut(
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_66097/4109112118.py:18: FutureWarning: The default of observed=False is deprecated and wil

alt.Chart(...)

### Location Type

In [ ]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V20111'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

V20111    1     2     3     4     5     6     7     8     9     10  ...    49  \
V20201                                                              ...         
12      1.01  0.30  1.44  1.09  2.01  0.53  1.73  1.26  1.46  0.71  ...  0.01   
14      2.87  0.29  1.69  0.44  2.06  0.00  3.68  1.55  1.91  0.74  ...  0.00   
33      1.35  0.36  1.77  1.14  2.55  0.57  3.75  1.40  1.30  0.42  ...  0.05   

V20111    50    51    52    53    54    55    56    57    58  
V20201                                                        
12      3.38  0.17  2.48  6.95  0.29  0.29  0.01  0.29  0.97  
14      5.30  0.00  2.43  2.50  0.07  0.81  0.00  0.22  1.47  
33      2.39  0.00  2.86  4.32  0.10  0.36  0.05  0.36  1.04  

[3 rows x 46 columns]


In [ ]:
cross_table_pct_reset = cross_table_pct.reset_index().melt(id_vars='V20201', var_name='V20111', value_name='percentage')

heatmap = alt.Chart(cross_table_pct_reset).mark_rect().encode(
    x=alt.X('V20111:N', title='Victim Race (V20111)'),
    y=alt.Y('V20201:N', title='Bias Motivation (V20201)'),
    color=alt.Color('percentage:Q', scale=alt.Scale(scheme='blues')),
    tooltip=['V20201', 'V20111', 'percentage']
).properties(
    width=300,
    height=400,
    title='Percentage of Victim Race by Bias Motivation'
)

heatmap

alt.Chart(...)

## Offender

### Offender Sex

In [78]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V50081'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

V50081           -7      0      1
V20201                           
12      32.96  1.17  11.74  54.14
14      31.30  1.20  13.80  53.69
32      19.83  0.42  16.81  62.93


### Offender Race

In [76]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V50091'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

V50091           -7      1      2     3     4     5
V20201                                             
12      28.05  8.65  53.94   7.60  0.65  0.96  0.14
14      27.29  9.95  40.69  18.38  0.32  2.97  0.40
32      16.66  6.80  57.85  16.70  0.99  0.80  0.19


### Offender Ethnicity

In [77]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V50111'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

V50111             0     1
V20201                    
12      77.59  19.32  3.08
14      70.14  25.04  4.82
32      66.76  28.09  5.16


### Offender Age

In [80]:
# Convert non-numeric values (e.g., strings) to NaN and then to numeric
race_df['V50071'] = pd.to_numeric(race_df['V50071'], errors='coerce')

# Drop rows with NaN in the age column (exclude missing age data from analysis)
race_df_clean = race_df.dropna(subset=['V50071'])

# Group age into 10-year bins
race_df_clean['age_group'] = pd.cut(
    race_df_clean['V50071'],
    bins=range(0, 101, 10),
    right=False,
    labels=['0-9','10-19','20-29','30-39','40-49',
            '50-59','60-69','70-79','80-89','90-99']
)

# Group by age_group and V20201, then reshape into long format
age_group_counts = (
    race_df_clean.groupby(['V20201', 'age_group'])
    .size()
    .reset_index(name='count')
)

# Calculate the percentage within each Bias Motivation group
age_group_counts['percentage'] = (
    age_group_counts.groupby('V20201')['count']
    .transform(lambda x: x / x.sum() * 100)
)

# Plot with Altair
import altair as alt

chart = alt.Chart(age_group_counts).mark_bar().encode(
    x=alt.X('age_group:N', title='Age Group'),
    y=alt.Y('percentage:Q', title='Percentage (%)'),
    color=alt.Color('age_group:N', legend=None),
    column=alt.Column('V20201:N', title='Bias Motivation (V20201)')
).properties(
    width=100,
    height=300,
    title='Age Distribution by Bias Motivation'
)

chart

/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_66097/992770572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_df['V50071'] = pd.to_numeric(race_df['V50071'], errors='coerce')
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_66097/992770572.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_df_clean['age_group'] = pd.cut(
/var/folders/53/wplxp6gd7z52xg76cg27rl3c0000gn/T/ipykernel_66097/992770572.py:18: FutureWarning: The default of observed=False is deprecated and will b

alt.Chart(...)

### RELATIONSHIP VIC TO OFF 1-1

In [ ]:
# Cross-tabulation (showing percentages)
cross_table_pct = pd.crosstab(race_df['V20201'], race_df['V40321'], normalize='index') * 100

# Round to two decimal places
print(cross_table_pct.round(2))

V40321    -9    -7    -6     1     3     4     5     8     9    10  ...  \
V20201                                                              ...   
12      0.01  2.66  3.93  0.01  0.01  0.05  0.05  0.01  0.01  0.02  ...   
14      0.00  1.25  2.50  0.07  0.00  0.07  0.07  0.00  0.00  0.00  ...   
32      0.00  2.55  2.18  0.03  0.03  0.03  0.03  0.00  0.00  0.00  ...   

V40321     25    26    27     3     4     5     6     7     8     9  
V20201                                                               
12      17.56  0.31  2.49  0.17  0.08  0.11  0.01  0.02  0.10  0.04  
14      27.30  0.00  4.71  0.00  0.00  0.00  0.00  0.00  0.07  0.00  
32      25.57  0.13  4.19  0.13  0.03  0.27  0.00  0.03  0.10  0.03  

[3 rows x 53 columns]


In [103]:
cross_table_pct_reset = cross_table_pct.reset_index().melt(id_vars='V20201', var_name='V40321', value_name='percentage')

heatmap = alt.Chart(cross_table_pct_reset).mark_rect().encode(
    x=alt.X('V40321:N', title='V40321'),
    y=alt.Y('V20201:N', title='Bias Motivation (V20201)'),
    color=alt.Color('percentage:Q', scale=alt.Scale(scheme='blues')),
    tooltip=['V20201', 'V40321', 'percentage']
).properties(
    width=300,
    height=400,
    title='Percentage of Victim Race by Bias Motivation'
)

heatmap

alt.Chart(...)